In [1]:
import os
import glob
import psycopg2
import pandas as pd
from ipynb.fs.full.sql_queries import *


def process_song_file(cur, filepath):
    """Processes a song file.
    
    Artist information is inserted into the 'artists' table. And Song information
    is inserted into the 'songs' table.
    
    Args:
        cur (psycopg2.cursor): A database cursor
        filepath (str): A filepath to a song file
    """
    # open song file
    df = pd.read_json(filepath, lines=True)
    
    for index, row in df.iterrows():
        # insert song record
        song_data = (row.song_id, row.title, row.artist_id, row.year, row.duration)
        try:
            cur.execute(song_table_insert, song_data)
        except psycopg2.Error as e:
            print("Error: Inserting row for table: songs")
            print (e)
    
        # insert artist record
        artist_data = (row.artist_id, row.artist_name, row.artist_location, row.artist_latitude, row.artist_longitude)
        try:
            cur.execute(artist_table_insert, artist_data)
        except psycopg2.Error as e:
            print("Error: Inserting row for table: artists")
            print (e)


def process_log_file(cur, filepath):
    """Processes a log file.
    
    Time information is inserted into the 'time' table. User information
    is upserted into the 'users' table. And Songplay information is
    inserted into the 'songplays' table.
    
    Args:
        cur (psycopg2.cursor): A database cursor
        filepath (str): A filepath to a log file
    """
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df.page == 'NextSong']

    # convert timestamp column to datetime
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    t = df.copy()
    
    # insert time data records
    time_data = (t.ts, t.ts.dt.hour , t.ts.dt.day , t.ts.dt.dayofweek , t.ts.dt.month , t.ts.dt.year , t.ts.dt.weekday)
    column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    time_df = pd.DataFrame(columns=column_labels)

    # it would be far more memory efficient to skip the time_data array and directly assign the 'dt' attributes to the time_df DataFrame
    for index, column_label in enumerate(column_labels):
        time_df[column_label] = time_data[index]

    for i, row in time_df.iterrows():
        try:
            cur.execute(time_table_insert, list(row))
        except psycopg2.Error as e:
            print("Error: Inserting row for table: time")
            print (e)

    # load user table
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    # insert user records
    for i, row in user_df.iterrows():
        try:
            cur.execute(user_table_insert, row)
        except psycopg2.Error as e:
            print("Error: Inserting row for table: users")
            print (e)
            
 # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        try:
            cur.execute(song_select, (row.song, row.artist, row.length))
            results = cur.fetchone()
        
            if results:
                songid, artistid = results
            else:
                songid, artistid = None, None

            # insert songplay record
            songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
            try:
                cur.execute(songplay_table_insert, songplay_data)
            except psycopg2.Error as e:
                print("Error: Inserting row for table: songplays")
                print (e)
        except psycopg2.Error as e:
            print("Error: Querying for Song ID and Artist ID")
            print (e)


def process_data(cur, conn, filepath, func):
    """Processes JSON files for a data directory path.
    
    Valid function values can be 'process_song_file' or
    'process_log_file'.
    
    Args:
        cur (psycopg2.cursor): A database cursor
        conn (psycopg2.connection): A database connection
        filepath (str): A filepath of the directory to process
        func (function): The function to call for each found file
    """
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))


def main():
    """Script main method.
    
    Creates a database connection, processes Song and Log information, and then
    closes the cursor and database connection.
    """
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
    except psycopg2.Error as e:
        print("Error: Could not make connection to the Postgres database")
        print(e)
    
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("Error: Could not get curser to the Database")
        print(e)
    
    process_data(cur, conn, filepath=r'C:\Users\SD-16\Documents\Practice_data\music_data_udacity\song_data', func=process_song_file)
    process_data(cur, conn, filepath=r'C:\Users\SD-16\Documents\Practice_data\music_data_udacity\log_data', func=process_log_file)

    cur.close()
    conn.close()


if __name__ == "__main__":
    main()

72 files found in C:\Users\SD-16\Documents\Practice_data\music_data_udacity\song_data
1/72 files processed.
2/72 files processed.
3/72 files processed.
4/72 files processed.
5/72 files processed.
6/72 files processed.
7/72 files processed.
8/72 files processed.
9/72 files processed.
10/72 files processed.
11/72 files processed.
12/72 files processed.
13/72 files processed.
14/72 files processed.
15/72 files processed.
16/72 files processed.
17/72 files processed.
18/72 files processed.
19/72 files processed.
20/72 files processed.
21/72 files processed.
22/72 files processed.
23/72 files processed.
24/72 files processed.
25/72 files processed.
26/72 files processed.
27/72 files processed.
28/72 files processed.
29/72 files processed.
30/72 files processed.
31/72 files processed.
32/72 files processed.
33/72 files processed.
34/72 files processed.
35/72 files processed.
36/72 files processed.
37/72 files processed.
38/72 files processed.
39/72 files processed.
40/72 files processed.
41/